# Исследование надежности заемщиков

Исследование проводится для кредитного отдела банка на основании статистики о платежеспособности клиентов. Результаты исследования будут учитываться при посторении модели кредитного скоринга. Для этого нам нужно достичь целей исследования.

**Цель исследования - проверка гипотез:**
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о поведении пользователей мы получаем из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы исправим самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.


### Шаг 1. Обзор данных

In [1]:
# импорт библиотеки pandas
# чтение файла с данными и сохранение в df
import pandas as pd
df = pd.read_csv('data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# получение общей информации о данных таблицы df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице содержится 12 столбцов. 2 столбца типа `float64`, 5 - `int64` и 5 - `object`.

Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Названия колонок не содержат ошибок. Количество значений в столбцах различается, значит в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные платежеспособности клиентов. Часть колонок описывает самого клиента :  количество детей в семье, общий трудовой стаж в днях, возраст клиента в годах, уровень образования клиента, семейное положение, пол клиента, тип занятости, имел ли задолженность по возврату кредитов, ежемесячный доход и цель получения кредита. Остальные данные - идентификаторы: идентификатор уровня образования, идентификатор семейного положения.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски, артефакты и дубликаты в данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Шаг 2.1 Заполнение пропусков

In [3]:
# подсчет пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
# подсчет процента пропусков от общего количества
display(pd.DataFrame(round((df.isna().mean()*100),2), columns=['NaNs, %']).style.format('{:.2f}').background_gradient('coolwarm'))

,"NaNs, %"
children,0.00
days_employed,10.10
dob_years,0.00
education,0.00
education_id,0.00
family_status,0.00
family_status_id,0.00
gender,0.00
income_type,0.00
debt,0.00


In [5]:
# получение первых 10 строк с пропусками в total_income
df[df['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Мы видим пропущенные значения в `days_employed` и `total_income`. Пропущенные значения составляют всего 10,1% от всех значенй в столбцах соотвественно. Можно предположить, что в нашем случае пропуски неслучайны, тк совпадают в обоих столбцах.

Для заполнения пропусков в `total_income` воспользуемся медианным значением. Использование такого метода предпочтительнее, так как среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются из большиства. 

In [6]:
# замена пропусков в total_income медианными значениями по группировке income_type
# получение первых 10 из заменяных строк для проверки
df['total_income'] = df.groupby(['income_type'])['total_income'].apply(lambda x: x.fillna(x.median()))
df[df['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,118514.486412,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447.935283,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,118514.486412,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,172357.950966,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,118514.486412,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,150447.935283,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,142594.396847,жилье


In [7]:
# проверка, подсчет пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

В значенияз столбца `days_employed` мы заметили аномалии, такие как отрицательные значения в трудовом стаже. Такие данные не отражают дейсвтительность и появились по системной ошибке. Заменим эти данные на их модули. 



Так же в этом столбце мы ранее увидели пропуски. Заменим их на медианное значение.

In [8]:
# замена отрицательных значений их модулями
# замена аномальных значений
# замена пропусков медианными значениями
# получение первых 10 строк таблицы df
df['days_employed'] = abs(df['days_employed'])
df.loc[df.loc[:,'income_type'] == 'пенсионер', 'days_employed'] = None
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()) 
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,1630.176471,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [9]:
# проверка на наличие пропусков
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверим значения на наличие аномалий в других столбцах.

In [10]:
# получение уникальных значений в столбце children
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы видим, что встречаются аномальные значения: количество детей не может быть отрицательным или сверхбольшим. Скорее всего значение 20 возникло в результате пропуска или ошибки, рассмотрим вариант, когда вместо 20 имелось ввиду значение 2.

In [11]:
# замена значений на их модули
# замена аномальных значений
# проверка уникальных значений 
df['children'] = abs(df['children'])
df.loc[df.loc[:, 'children'] == 20, 'children'] = 2  
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [12]:
# проверка значений в столбце dob_years
df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [13]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [14]:
df[df['dob_years'] == 0].sample(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6670,0,1630.176471,0,Высшее,0,в разводе,3,F,пенсионер,0,118514.486412,покупка жилой недвижимости
8061,0,1630.176471,0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,61804.271999,высшее образование
18851,0,10637.001569,0,среднее,1,женат / замужем,0,F,госслужащий,0,136537.132625,дополнительное образование
6407,2,1653.972845,0,среднее,1,женат / замужем,0,F,сотрудник,0,181510.868867,жилье
4147,0,3549.117333,0,среднее,1,в разводе,3,M,сотрудник,0,108130.933212,покупка жилой недвижимости
19829,0,1630.176471,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
2870,0,1875.972013,0,среднее,1,женат / замужем,0,F,компаньон,0,167713.212064,на покупку своего автомобиля
6848,2,1146.689586,0,среднее,1,женат / замужем,0,F,сотрудник,0,101545.049355,приобретение автомобиля
18176,0,3533.403832,0,Среднее,1,гражданский брак,1,F,сотрудник,1,110485.456043,свадьба
3044,2,2502.891798,0,высшее,0,Не женат / не замужем,4,M,сотрудник,0,292145.267076,операции с недвижимостью


Мы нашли еще одну аномалию: 0 в значениях возраста заемщика. Будем считать это за пропуск. Видим, что пропуски в этом столбце никак не связаны с другими значениями. Всего таких 101 значение по отношению к 21525, что составляет 0,4 процента. Поэтому эти пропуски мы удалим.

In [15]:
# удаление строк с пропущенными значениями
df['dob_years'] = df['dob_years'].replace(0, 'Nan')
df['dob_years'] = df['dob_years'].dropna()

In [16]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [17]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [18]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

В столбцах 'education_id', 'family_status_id' и 'debt' все выглядит достоверно.

In [19]:
# подсчет уникальных значений
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В данном исследование пол не влияет на результаты. Мы можем удалить эту строку или же заменить значение на какой-либо гендер.

In [20]:
df['gender'] = df['gender'].replace('XNA', 'M')

In [21]:
# проверка
df['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

Мы разобрались с аномальными значениями. Дальше по необходимости заменим типы данных и удалим дубликаты.

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных `float` на целочисленный `int` в столбцах `total_income` и `days_emploued`,  так как количество дней и доход могут иметь только целое значение.

In [22]:
# замена типа данных в столбцах total_income и days_employed
df['total_income'] = df['total_income'].astype(int)
df['days_employed'] = df['days_employed'].astype(int)

### Шаг 2.4. Удаление дубликатов.

В данных встречаются явные дубликаты. Удалим их с помощью метода `drop_duplicates`. В столбце `education` встречаются дубликаты по регистру, для их удаления воспользуемся методом `str.lower()`.

In [23]:
# подсчет дубликатов в df
df.duplicated().sum()

54

In [24]:
# удаления явных дубликатов
df = df.drop_duplicates().reset_index(drop=True)

In [25]:
# проверка
df.duplicated().sum()

0

In [26]:
# подсчет уникальных значений в стобце education с помощью метода value_counts()
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [27]:
# приведение всех значений столбца к нижнему регистру методом str.lower()
df['education'] = df['education'].str.lower()

In [28]:
# подсчет уникальных значений в стобце education после приведение к нижнему регистру
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим остальные столбцы на дубликаты

In [29]:
# подсчет уникальных значений
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [30]:
# приведение значений столбца к нижнему регистру
df['family_status'] = df['family_status'].str.lower()

In [31]:
# получение уникальных значений
df['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [32]:
# получение уникальных значений
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [33]:
df.duplicated().sum()

17

После стандартизации написания появились полные дубликаты. Удалим их.

In [34]:
df = df.drop_duplicates().reset_index(drop=True)

In [35]:
df.duplicated().sum()

0

В столбце `purpose` мы видим дубликаты, такие как `свадьба`, `на проведение свадьбы`, `сыграть свадьбу`. Удалять их сейчас не будем, так как потом произведем категоризацию.

 **Вывод**
 
Таким образом мы избавились от дубликтов в данных. Сначала мы удалили явные дубликаты методом `drop_duplicates()`, значения в ячейках которых полностью совпадали. Такие дубликаты могут возникнуть по техничсекой ошибке, например, при выгрузке данных или при ручном заполнении, когда человек случайно дважды внес одни и те же данные.

Далее мы удалили дубликаты по регистру с помощью метода `str.lower()`.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых: 
* каждому уникальному значению из `education` соответствует уникальное значение `education_id` — в первом;
* каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id` — во втором.

Далее мы сможем обращаться к новым датафреймам по идентификаторам.

In [36]:
# запишем новый датафрейм education
data_education = df[['education_id', 'education']]

In [37]:
# запишем новый датафрейм family_status
data_family = df[['family_status_id', 'family_status']]

In [38]:
# удаление дубликатов в новых датафреймах
data_education = data_education.drop_duplicates().reset_index(drop=True)
data_family = data_family.drop_duplicates().reset_index(drop=True)

In [39]:
# удаление столбцов 'family_status', 'education' из старого датафрейма и запись его в df
df = df.drop(columns=['family_status', 'education'], axis=0)
df.sample(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
12107,0,3441,34,0,0,F,госслужащий,0,260372,операции со своей недвижимостью
16458,2,3015,42,0,0,F,госслужащий,0,66459,образование
3337,0,1630,60,1,4,F,пенсионер,0,97511,сделка с автомобилем
15526,3,1041,30,1,0,F,сотрудник,0,111726,покупка своего жилья
10278,0,1630,62,0,2,F,пенсионер,0,249382,операции с недвижимостью
7677,0,173,45,1,4,M,сотрудник,1,286002,профильное образование
13226,1,4691,40,0,1,F,сотрудник,0,117124,на проведение свадьбы
1509,1,9184,43,1,0,F,госслужащий,0,101771,образование
19768,0,1329,51,1,0,F,компаньон,0,160873,высшее образование
11801,0,375,32,0,3,F,сотрудник,0,192991,покупка коммерческой недвижимости


### Шаг 2.6. Категоризация дохода.

Для утобства создадим столбец `total_income_category`, в котором распределим кредитополучателей по категории в зависимости от величины дохода:
* 0–30000 — `'E'`;
* 30001–50000 — `'D'`;
* 50001–200000 — `'C'`;
* 200001–1000000 — `'B'`;
* 1000001 и выше — `'A'`.

In [40]:
# создание функции total_income_category для определения диапазонов категорий
# применение фукнции с столбцу 'total_income' и запись результатов в новый столбец 'total_income_category'
# получение первых 10 строк таблицы df_new
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income >= 30001 and total_income <= 50000:
        return 'D'
    if total_income >= 50001 and total_income <= 200000:
        return 'C'
    if total_income >= 200001 and total_income <= 1000000:
        return 'B'
    if total_income >= 1000001:
        return 'A'
df['total_income_category'] = df['total_income'].apply(total_income_category)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,1630,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Для категоризации целей кредита мы выявили ключевые слова, которые будет искать условная конструкция `if`.

In [41]:
# создание функции purpose_category для определения категорий
# применение фукнции с столбцу 'purpose' и запись результатов в новый столбец 'purpose_category'
# получение первых 10 строк таблицы df_new
def purpose_category(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жил' or 'недвиж' in purpose:
        return 'операции с недвижимостью'
df['purpose_category'] = df['purpose'].apply(purpose_category)  
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,1630,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [42]:
def correlation(col):
    pivot = df.pivot_table(index=col, columns='debt', aggfunc='size')
    pivot['%'] = round(pivot[1] / (pivot[0] + pivot[1]) * 100, 2)
    return pivot

In [43]:
correlation('children')

debt,0,1,%
children,,,
0,13028.0,1063.0,7.54
1,4410.0,445.0,9.17
2,1926.0,202.0,9.49
3,303.0,27.0,8.18
4,37.0,4.0,9.76
5,9.0,NaN,NaN


##### Вывод 1:

Изучив сводную таблицу можно сделать вывод, что в целом количество детей не сильно влияет на возврат кредита в срок. Семьи с одним ребенком имеют не намного больше задолженностей по старым кредитам, чем семьи без детей. В тоже время семьи с тремя детьми практически не отличаются по показаетлю от семей без детей. Семьи с 5 детьми не имеют задолженностей по кредитам. Так же можно заметить, что семьи без детей чаще берут кредиты.

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
correlation('family_status_id')

debt,0,1,%
family_status_id,,,
0,11408,931,7.55
1,3763,388,9.35
2,896,63,6.57
3,1110,85,7.11
4,2536,274,9.75


##### Вывод 2:

Изучив сводную таблицу можно сделать вывод, что люди не состоящие в браке или сожители, просрочивают кредиты чаще других категорий, но незначительно. Так же можно увидеть, что люди состоящие в браке берут кредиты намного чаще остальных.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
correlation('total_income_category')

debt,0,1,%
total_income_category,,,
A,23,2,8.00
B,4686,356,7.06
C,14655,1360,8.49
D,329,21,6.00
E,20,2,9.09


##### Вывод 3:

Мы видим, что люди с низким доходом в категории E чаще просрочивают выплаты по кредитам. Тем не менее люди из категории D реже имеют задолженности по старым кредитам, в отличии от людей с более высоким доходом. Так же можно сделать вывод, что чаще всего берут кредиты люди из категории C.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [46]:
correlation('purpose_category')

debt,0,1,%
purpose_category,,,
операции с автомобилем,3903,403,9.36
операции с недвижимостью,10029,782,7.23
получение образования,3643,370,9.22
проведение свадьбы,2138,186,8.00


##### Вывод 4:

Изучив сводную таблицу, мы можем сделать вывод, что чаще просрочивают выплату кредитов на операции с автомобилями и на получение образования.

## Общий вывод:

По итогам исследования, мы ответили на поставленные вопросы и сделали несколько выводов. 

Мы выяснили, что наличие детей не влияет на погашение кредита в срок. Тем не менее по данным свобдной таблицы мы увидели, что семьи с 1, 2 и 4 детьми чаще задерживают выплаты по кредитам. Так же мы выяснили, что семьи без детей берут кредиты в несколько раз чаще семей с детьми. 

Далее мы изучили зависимость между семейным положением и возвратом кредита в срок. Мы увидели, что люди не состоящие в браке или сожители (гражданский брак) просрочивают выплаты чаще остальных. Люди состоящие в браке берут кредиты намного чаще.

Изучив зависимость между уровнем дохода и возвратом кредита в срок, мы выяснили, что люди зарабатывающие меньше 30000 просрочивают кредиты чаще других категорий. Люди с доходом от 30000 до 50000 реже всего имеют долги по кредитам. Так же чаще всего берут кредиты люди с доходом от 50000 до 200000.

В конце исследования нам стало извсетно, что чаще задерживают выплаты по кредитам на операции с автомобилями и на получение образования. Чаще всего берутся кредиты на операции с недвижимостью, на эти цели просрочки по кредитам встречаются реже всего.